In [18]:
import llm2geneset
import json
import time
import openai
import numpy as np
import pandas as pd

In [2]:
aclient = openai.AsyncClient()

In [3]:
cond1 = "parkinson's disease"
cond2 = "matched controls"

pathways = await llm2geneset.get_pathways(aclient, cond1, cond2, n=50)

100%|█████████████████████████████████████████████| 1/1 [00:06<00:00,  6.41s/it]


In [4]:
len(pathways)

95

In [14]:
pathways[12]

np.str_('Cell-cycle regulation')

In [6]:
queries = [p+f" in {cond1}" for p in pathways]  

In [7]:
pmids = await llm2geneset.esearch_async(queries, "pubmed", 1)

100%|███████████████████████████████████████████| 95/95 [00:47<00:00,  2.01it/s]


In [8]:
pmids

[['34691027'],
 ['35447272'],
 ['35390677'],
 ['37892117'],
 ['30590010'],
 ['33589594'],
 ['31999942'],
 ['32726602'],
 ['36152810'],
 ['24252804'],
 ['34389279'],
 ['35435793'],
 ['35435793'],
 ['35260343'],
 ['31282427'],
 ['34729301'],
 ['36569603'],
 ['34784980'],
 ['33182554'],
 ['25553450'],
 ['37647388'],
 ['34389279'],
 ['34389279'],
 ['21303898'],
 ['37716354'],
 ['32048886'],
 ['26860955'],
 ['31583594'],
 ['22355802'],
 ['32854418'],
 ['32726602'],
 ['21303898'],
 ['37819757'],
 ['25124148'],
 ['25462288'],
 ['34695538'],
 ['34695538'],
 ['37351278'],
 ['32674367'],
 ['35848038'],
 ['32092326'],
 ['34739911'],
 ['35260343'],
 ['34739911'],
 ['35551349'],
 ['32576618'],
 ['37568205'],
 ['34691027'],
 ['27757833'],
 ['33321831'],
 ['23643800'],
 ['23643800'],
 ['23643800'],
 ['23643800'],
 ['34691027'],
 ['33894193'],
 ['31607362'],
 ['37048085'],
 ['33297340'],
 ['33182554'],
 ['31686637'],
 ['36839332'],
 ['34979259'],
 ['37210012'],
 ['31282427'],
 ['35910843'],
 ['3643657

In [17]:
# have repeated pmids
# pathways that from the same pmid likely is the same pathway but with slight discription difference
# need to delete the repeated pathways
len(np.unique(pmids))

69

In [25]:
pathway_df = pd.DataFrame({'pathway':pathways, 'pmids':[i[0] for i in pmids]})
pathway_df.head()

,pathway,pmids
0,Alpha-synuclein signaling,34691027
1,Amyloid precursor protein catabolic process,35447272
2,Apoptosis regulation,35390677
3,Autophagy,37892117
4,Autophagy-lysosome pathway,30590010


In [30]:
pathway_df.groupby('pmids').count().reset_index().sort_values('pathway', ascending=False)

,pmids,pathway
4,23643800,6
28,32726602,4
1,21303898,3
37,34389279,3
39,34691027,3
...,...,...
63,37351278,1
65,37647388,1
66,37716354,1
67,37819757,1


In [32]:
pathway_df[pathway_df.pmids=='23643800']

,pathway,pmids
50,Mitochondrial dynamics,23643800
51,Mitochondrial dysfunction,23643800
52,Mitochondrial electron transport chain,23643800
53,Mitochondrial fission,23643800
69,Programmed cell death,23643800
76,Reactive oxygen species metabolic process,23643800


In [9]:
abstracts = await llm2geneset.efetch_pubmed_async(pmids)

100%|███████████████████████████████████████████| 95/95 [00:47<00:00,  2.01it/s]


In [10]:
abstracts[0:3]

[[{'pmid': '34691027',
   'title': "Targeting Microglial α-Synuclein/TLRs/NF-kappaB/NLRP3 Inflammasome Axis in Parkinson's Disease.",
   'abstract': " According to emerging studies, the excessive activation of microglia and the subsequent release of pro-inflammatory cytokines play important roles in the pathogenesis and progression of Parkinson's disease (PD). However, the exact mechanisms governing chronic neuroinflammation remain elusive. Findings demonstrate an elevated level of NLRP3 inflammasome in activated microglia in the substantia nigra of PD patients. Activated NLRP3 inflammasome aggravates the pathology and accelerates the progression of neurodegenerative diseases. Abnormal protein aggregation of α-synuclein (α-syn), a pathologically relevant protein of PD, were reported to activate the NLRP3 inflammasome of microglia through interaction with toll-like receptors (TLRs). This eventually releases pro-inflammatory cytokines through the translocation of nuclear factor kappa-B (

In [9]:
abstracts[0][0]['title']

'Glycodelin and beta-lactoglobulin, lipocalins with a high structural similarity, differ in ligand binding properties.'

In [10]:
abstracts[0][0]['abstract']

' Human glycodelin, a lipocalin with a high amino acid similarity to beta-lactoglobulins, appears as various glycoforms with different biological activities in endometrium (glycodelin-A) and seminal plasma (glycodelin-S). We found that the structures of these glycodelins and beta-lactoglobulin are similar. Despite this structural similarity, unlike beta-lactoglobulin, glycodelin-A binds neither retinoic acid nor retinol. It was impossible to detect any endogenous retinoids or steroids in any of the two purified glycodelins. Both their glycoforms share similar thermodynamic parameters of reversible denaturation suggesting that native folding of glycodelin-A and glycodelin-S is not influenced by the differences in glycosylation or by ligand binding.'

In [ ]:

        abstracts = await llm2geneset.efetch_pubmed_async(pmids)
        end_time = time.time()
        gen_res["single_relevant_abstract"] = abstracts
        gen_res["single_relevant_abstract_searchtime"] = end_time - start_time
        with open('libs_human/' + model + '/' + lib_name + '.json', 'w') as json_file:
            json.dump(gen_res, json_file, indent=4)
